## Data Set Construction

**Functions**

`pd.read_csv`, `pd.read_excel`, `np.diff` or `DataFrame.diff`, `DataFrame.resample`

### Exercise 1

1. Download all available daily data for the S&P 500 and the Hang Seng Index from Yahoo! Finance. 
2. Import both data sets into Python. The final dataset should have a `DateTimeIndex`, and the date
   column should not be part of the `DataFrame`.
3. Construct weekly price series from each, using Tuesday prices (less likely to be a holiday).
4. Construct monthly price series from each using last day in the month.
5. Save the data to the HDF file "equity-indices.h5".


Let's go!

In [16]:
import pandas as pd
import numpy as np
import scipy as scipy

In [17]:
sp500 = pd.read_csv("data/GSPC.csv",parse_dates=True,index_col="Date")
hsi = pd.read_csv("data/GSPC.csv", parse_dates=True, index_col="Date")

In [18]:
weekly_sp500 = sp500.resample("W-TUE").last()
weekly_hsi = hsi.resample("W-TUE").last()

In [19]:
monthly_sp500 = sp500.resample("M").last()
monthly_hsi = hsi.resample("M").last()

In [20]:
h5file = pd.HDFStore("data/equity-indices.h5", mode="w")

In [21]:
h5file.append("sp500", sp500)
h5file.append("hsi",hsi)
h5file.append("weekly_sp500", weekly_sp500)
h5file.append("weekly_hsi", weekly_hsi)
h5file.append("monthly_sp500", monthly_sp500)
h5file.append("monthly_hsi", monthly_hsi)
h5file.close()

In [22]:
sp500.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1950-01-03,16.66,16.66,16.66,16.66,16.66,1260000
1950-01-04,16.85,16.85,16.85,16.85,16.85,1890000
1950-01-05,16.93,16.93,16.93,16.93,16.93,2550000
1950-01-06,16.98,16.98,16.98,16.98,16.98,2010000
1950-01-09,17.08,17.08,17.08,17.08,17.08,2520000


In [7]:
weekly_sp500.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-08-27,2893.139893,2898.790039,2860.590088,2869.159912,2869.159912,3533630000
2019-09-03,2909.010010,2914.389893,2891.850098,2906.270020,2906.270020,3426790000
2019-09-10,2971.010010,2979.389893,2957.010010,2979.389893,2979.389893,4390770000
2019-09-17,2995.669922,3006.209961,2993.729980,3005.699951,3005.699951,3671840000
2019-09-24,2983.500000,2999.149902,2982.229980,2991.780029,2991.780029,3186590000


In [8]:
monthly_sp500.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-05-31,2766.149902,2768.979980,2750.520020,2752.060059,2752.060059,3981020000
2019-06-30,2932.939941,2943.979980,2929.050049,2941.760010,2941.760010,5420700000
2019-07-31,3016.219971,3017.399902,2958.080078,2980.379883,2980.379883,4623430000
2019-08-31,2937.090088,2940.429932,2913.320068,2926.459961,2926.459961,3008450000
2019-09-30,2983.500000,2999.149902,2982.229980,2991.780029,2991.780029,3186590000


### Exercise 2

Write a function that will correctly aggregate to weekly or monthly respecting the
aggregation rules

* High: `max`
* Low: `min`
* Volume: `sum`

The signature should be:

```python
def yahoo_agg(data, freq):
    <code here>
    return resampled_data
```


In [26]:
def yahoo_agg(data, freq):
    resampler = data.resample(freq)

    high = resampler.High.max()
    low = resampler.Low.min()
    vol = resampler.Volume.sum()
    #start with last for all columns
    resampled_data = resampler.last()
    resampled_data["High"] = high
    resampled_data["Low"] = low
    resampled_data["Volume"] = vol

    return resampled_data

In [31]:
better_monthly_sp500 = yahoo_agg(sp500,"M")
better_monthly_sp500.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-05-31,2766.149902,2954.129883,2750.520020,2752.060059,2752.060059,76860120000
2019-06-30,2932.939941,2964.149902,2728.810059,2941.760010,2941.760010,70881390000
2019-07-31,3016.219971,3027.979980,2952.219971,2980.379883,2980.379883,70349470000
2019-08-31,2937.090088,3013.590088,2822.120117,2926.459961,2926.459961,79599440000
2019-09-30,2983.500000,3021.989990,2891.850098,2991.780029,2991.780029,57236800000


In [32]:
monthly_sp500.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-05-31,2766.149902,2768.979980,2750.520020,2752.060059,2752.060059,3981020000
2019-06-30,2932.939941,2943.979980,2929.050049,2941.760010,2941.760010,5420700000
2019-07-31,3016.219971,3017.399902,2958.080078,2980.379883,2980.379883,4623430000
2019-08-31,2937.090088,2940.429932,2913.320068,2926.459961,2926.459961,3008450000
2019-09-30,2983.500000,2999.149902,2982.229980,2991.780029,2991.780029,3186590000


### Exercise 3

1. Import the Fama-French benchmark portfolios as well as the 25 sorted portfolios at both the
   monthly and daily horizon from [Ken French"s Data Library](http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html).
   **Note** It is much easier to clean to data file before importing than to find the precise
   command that will load the unmodified data.
2. Import daily FX rate data for USD against AUD, Euro, JPY and GBP from the [Federal Reserve Economic Database (FRED)](http://research.stlouisfed.org/fred2/categories/94). Use Excel rather than csv files.
3. Save the data to the HDF files "fama-french.h5" and "fx.h5"

In [34]:
yen_dollar = pd.read_excel("data/DEXJPUS.xls", index_col = "observation_date", skiprows=10)
dollar_aud = pd.read_excel("data/DEXUSAL.xls", index_col = "observation_date", skiprows=10)
dollar_euro = pd.read_excel("data/DEXUSEU.xls", index_col = "observation_date", skiprows=10)
dollar_pound = pd.read_excel("data/DEXUSUK.xls", index_col = "observation_date", skiprows=10)

In [35]:
fx = pd.concat([yen_dollar, dollar_aud, dollar_euro, dollar_pound], axis=1)
print(fx.tail())

                  DEXJPUS  DEXUSAL  DEXUSEU  DEXUSUK
observation_date                                    
2019-09-16         107.94   0.6865   1.1006   1.2423
2019-09-17         108.12   0.6858   1.1063   1.2493
2019-09-18         108.17   0.6844   1.1061   1.2478
2019-09-19         108.00   0.6795   1.1046   1.2481
2019-09-20         107.95   0.6770   1.1004   1.2489


In [37]:
fx.to_hdf("data/fx.h5", "fx")

In [59]:
ff_5x5 = pd.read_csv("data/25_Portfolios_5x5.CSV",index_col=0)
ff_factors = pd.read_csv("data/F-F_Research_Data_Factors.CSV", index_col=0)
ff = pd.concat([ff_factors, ff_5x5],axis =1)


In [60]:
ff

,Mkt-RF,SMB,HML,RF,SMALL LoBM,ME1 BM2,ME1 BM3,ME1 BM4,SMALL HiBM,ME2 BM1,...,ME4 BM1,ME4 BM2,ME4 BM3,ME4 BM4,ME4 BM5,BIG LoBM,ME5 BM2,ME5 BM3,ME5 BM4,BIG HiBM
192607,2.96,-2.30,-2.87,0.22,3.7782,-0.4119,-1.9434,0.3530,2.0534,2.1904,...,1.5893,1.5278,1.1869,0.2727,2.4678,3.4539,6.0902,2.0266,3.1111,0.5623
192608,2.64,-1.40,4.19,0.25,-2.2074,-8.7275,2.4404,0.6086,8.3968,2.1709,...,1.3336,3.8730,2.0059,2.1706,5.3422,1.0124,4.1903,2.0131,5.4849,7.7576
192609,0.36,-1.32,0.01,0.23,-6.2113,-0.2989,-6.1982,-1.6368,0.8649,-1.8550,...,1.0923,-0.5250,-1.7314,1.4646,0.8730,-1.2906,3.6538,0.0950,-0.7487,-2.4284
192610,-3.24,0.04,0.51,0.32,-8.6241,-3.7532,-5.6719,5.7170,-2.5476,-1.7995,...,-3.3361,-2.6559,-2.0316,-3.1051,-5.3525,-2.7413,-3.0071,-2.2437,-4.6719,-5.8129
192611,2.53,-0.20,-0.35,0.31,3.4744,6.6476,2.2634,-4.7020,0.5362,2.9051,...,3.4448,2.3887,3.7403,4.9320,1.8213,4.2946,2.5326,1.5204,3.6619,2.5636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201903,1.10,-3.13,-4.07,0.19,0.8891,-0.4743,-2.5100,-4.9485,-3.4235,-0.2319,...,1.8237,1.0998,-0.9002,-0.5395,-2.8702,3.3344,1.9283,-0.1602,-0.5929,-3.0620
201904,3.96,-1.68,1.93,0.21,0.6068,1.6518,2.1840,3.4912,2.0865,2.8361,...,3.1410,4.2046,4.3501,5.0762,4.3419,4.6039,2.9807,3.6429,5.1341,9.3863
201905,-6.94,-1.20,-2.39,0.21,-5.5586,-5.3126,-7.7829,-8.8025,-7.9767,-8.9190,...,-5.7334,-7.1667,-7.7153,-9.1637,-7.9580,-6.9663,-5.7641,-5.0654,-8.1314,-8.7586
201906,6.93,0.33,-1.08,0.18,7.6353,6.2398,4.7677,6.7187,4.7008,8.9242,...,7.2875,9.4043,9.5103,7.5115,6.9246,7.7424,5.8872,5.9640,6.6670,7.9347


In [63]:
dates = []
for value in ff.index:
    year = value // 100
    month = value % 100
    dates.append(pd.Timestamp(year=year, month=month, day=1))
ff.index = dates
ff.tail()    

,Mkt-RF,SMB,HML,RF,SMALL LoBM,ME1 BM2,ME1 BM3,ME1 BM4,SMALL HiBM,ME2 BM1,...,ME4 BM1,ME4 BM2,ME4 BM3,ME4 BM4,ME4 BM5,BIG LoBM,ME5 BM2,ME5 BM3,ME5 BM4,BIG HiBM
2019-03-01,1.10,-3.13,-4.07,0.19,0.8891,-0.4743,-2.5100,-4.9485,-3.4235,-0.2319,...,1.8237,1.0998,-0.9002,-0.5395,-2.8702,3.3344,1.9283,-0.1602,-0.5929,-3.0620
2019-04-01,3.96,-1.68,1.93,0.21,0.6068,1.6518,2.1840,3.4912,2.0865,2.8361,...,3.1410,4.2046,4.3501,5.0762,4.3419,4.6039,2.9807,3.6429,5.1341,9.3863
2019-05-01,-6.94,-1.20,-2.39,0.21,-5.5586,-5.3126,-7.7829,-8.8025,-7.9767,-8.9190,...,-5.7334,-7.1667,-7.7153,-9.1637,-7.9580,-6.9663,-5.7641,-5.0654,-8.1314,-8.7586
2019-06-01,6.93,0.33,-1.08,0.18,7.6353,6.2398,4.7677,6.7187,4.7008,8.9242,...,7.2875,9.4043,9.5103,7.5115,6.9246,7.7424,5.8872,5.9640,6.6670,7.9347
2019-07-01,1.19,-2.06,0.12,0.19,-3.8504,-1.1600,-0.8926,-1.6496,-2.5149,-3.1247,...,1.6768,1.0410,1.9332,-0.6625,0.9940,1.6565,1.9146,1.5096,0.9860,2.1159


In [64]:
ff = ff.resample("M").last()
ff.tail()
#trick to get the last day of the month - as we have only monthly data this doesn't change

,Mkt-RF,SMB,HML,RF,SMALL LoBM,ME1 BM2,ME1 BM3,ME1 BM4,SMALL HiBM,ME2 BM1,...,ME4 BM1,ME4 BM2,ME4 BM3,ME4 BM4,ME4 BM5,BIG LoBM,ME5 BM2,ME5 BM3,ME5 BM4,BIG HiBM
2019-03-31,1.10,-3.13,-4.07,0.19,0.8891,-0.4743,-2.5100,-4.9485,-3.4235,-0.2319,...,1.8237,1.0998,-0.9002,-0.5395,-2.8702,3.3344,1.9283,-0.1602,-0.5929,-3.0620
2019-04-30,3.96,-1.68,1.93,0.21,0.6068,1.6518,2.1840,3.4912,2.0865,2.8361,...,3.1410,4.2046,4.3501,5.0762,4.3419,4.6039,2.9807,3.6429,5.1341,9.3863
2019-05-31,-6.94,-1.20,-2.39,0.21,-5.5586,-5.3126,-7.7829,-8.8025,-7.9767,-8.9190,...,-5.7334,-7.1667,-7.7153,-9.1637,-7.9580,-6.9663,-5.7641,-5.0654,-8.1314,-8.7586
2019-06-30,6.93,0.33,-1.08,0.18,7.6353,6.2398,4.7677,6.7187,4.7008,8.9242,...,7.2875,9.4043,9.5103,7.5115,6.9246,7.7424,5.8872,5.9640,6.6670,7.9347
2019-07-31,1.19,-2.06,0.12,0.19,-3.8504,-1.1600,-0.8926,-1.6496,-2.5149,-3.1247,...,1.6768,1.0410,1.9332,-0.6625,0.9940,1.6565,1.9146,1.5096,0.9860,2.1159


In [65]:
ff.to_hdf("data/ff.h5","ff")

In [66]:
#exactly the same mothods to do the daily time series of ff data
type(ff.index)
#shows us what type of data we are dealing with

pandas.core.indexes.datetimes.DatetimeIndex

### Exercise 3 (Alternative method)

1. Install and use `pandas-datareader` to repeat the previous exercise.

#### Preliminary Step

You must first install the module using 

```
pip install pandas-datareader
``` 

from the command line. Then you can run this code. **Note**: Running this code requires access
to the internet.

In [67]:
%pip install pandas-datareader

     |████████████████████████████████| 107 kB 1.7 MB/s 
Note: you may need to restart the kernel to use updated packages.


In [69]:
import pandas_datareader as pdr

In [75]:
yen_dollar2 = pdr.get_data_fred("DEXJPUS", start = "1950")
yen_dollar2

,DEXJPUS
DATE,
1971-01-04,357.73
1971-01-05,357.81
1971-01-06,357.86
1971-01-07,357.87
1971-01-08,357.82
...,...
2020-11-09,105.58
2020-11-10,105.36
2020-11-11,NaN


In [80]:
yen_dollar_until_2000 = pdr.get_data_fred("DEXJPUS",start="1950", end="2000")
yen_dollar_until_2000

,DEXJPUS
DATE,
1971-01-04,357.73
1971-01-05,357.81
1971-01-06,357.86
1971-01-07,357.87
1971-01-08,357.82
...,...
1999-12-27,102.10
1999-12-28,102.19
1999-12-29,102.05


In [82]:
type(yen_dollar2.index)

pandas.core.indexes.datetimes.DatetimeIndex

### Exercise 4
Download data on 1 year and 10 year US government bond rates from FRED, and 
construct the term premium as the different in yields on 10 year and 1 year
bonds. Combine the two yield series and the term premium into a `DataFrame`
and save it as HDF.

In [84]:
gs10 = pdr.get_data_fred("GS10",start="1950")
gs1 = pdr.get_data_fred("GS1",start="1950")
gs10

,GS10
DATE,
1953-04-01,2.83
1953-05-01,3.05
1953-06-01,3.11
1953-07-01,2.93
1953-08-01,2.95
...,...
2020-06-01,0.73
2020-07-01,0.62
2020-08-01,0.65


In [85]:
gs1

,GS1
DATE,
1953-04-01,2.36
1953-05-01,2.48
1953-06-01,2.45
1953-07-01,2.38
1953-08-01,2.28
...,...
2020-06-01,0.18
2020-07-01,0.15
2020-08-01,0.13


In [86]:
gs1.shape
#returns a tuple with the row and column dimensions of the dataframe

(811, 1)

In [88]:
term = gs10["GS10"] - gs1["GS1"]
term.name = "TERM"
term

DATE
1953-04-01    0.47
1953-05-01    0.57
1953-06-01    0.66
1953-07-01    0.55
1953-08-01    0.67
              ... 
2020-06-01    0.55
2020-07-01    0.47
2020-08-01    0.52
2020-09-01    0.55
2020-10-01    0.66
Name: TERM, Length: 811, dtype: float64

In [91]:
combined = pd.DataFrame([term, gs10["GS10"], gs1["GS1"]]).T
combined.tail()

,TERM,GS10,GS1
DATE,,,
2020-06-01,0.55,0.73,0.18
2020-07-01,0.47,0.62,0.15
2020-08-01,0.52,0.65,0.13
2020-09-01,0.55,0.68,0.13
2020-10-01,0.66,0.79,0.13


In [92]:
combined.to_hdf("data/term-premium.h5","term_premium")